#### **온통청년** 정책 크롤링


##### 정책 ID 크롤링

- 목록 정렬: 60 개씩, 총 59 페이지
- 상시, 진행중 현황을 알 수 있는 태그가 있어 `24년 12월 23일 기준`으로 `상시`, `진행중`인 정책들만 추출함
  - ```
    <div class="badge">
      <span class="label green">진행중</span>
      <span class="cate">주거분야</span>
    </div>
    ```
- 각 정책의 링크가 아래와 같이 구성되어있었는데, `id` 부분의 `dtlLink_R2024011018628` 에서 `dtlLink_`를 제외한 부분이 각 정책의 코드번호
  - `<a href="#"id="dtlLink_R2024011018628"onclick="f_Detail('R2024011018628');"class="tit">청년 주택드림 청약통장</a>`
  - 따라서 1 페이지부터 59 페이지까지 페이지를 이동시키며 각 페이지에 있는 60개의 정책들의 `id`만 `list`로 추출


In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re


def get_ids_with_state(page_num: int, url: str):
    """
    온통청년의 상시, 진행중인 정책 ID를 추출하는 함수입니다.
    Parameters:
        page_num (int): 추출하려는 총 페이지 수
        url (str): 추출 대상의 페이지 링크를 조합할 베이스 링크
    Returns:
        policy_id_list (list): 상시, 진행중인 정책의 ID를 list로 모아 반환합니다.
    """
    policy_id_list = []
    # 1 - 59 페이지
    for i in range(1, page_num + 1):
        # URL 뒤에 페이지 index만 바뀌게
        URL = url
        URL += str(i)
        response = requests.get(URL)
        soup = bs(response.text, "lxml")
        # 상태를 추출할 badge
        badges = soup.select("div.badge")
        # 정책 제목 추출
        titles = soup.select("a.tit")
        organ = soup.select("div.organ-name")
        for j in range(len(titles)):
            # 정책 상태 추출
            badge = badges[j].find(name="span", attrs="label").text
            # 정책이 진행중, 혹은 상시일 경우
            if badge in ["진행중", "상시"]:
                # 정책 ID 추출
                policy_id = titles[j].attrs["id"].replace("dtlLink_", "")
                organ_name = organ[j].select_one("p")
                organ_name = re.sub(r"<.*?>", "", str(organ_name))
                if organ_name == "세종 세종":
                    organ_name = "세종"
                # list에 삽입시 중복 제거
                if policy_id not in policy_id_list:
                    policy_id_list.append([policy_id, organ_name])
    return policy_id_list

In [2]:
# 상시, 모집중 정책 ID/기관명 크롤링
policy_id_list = []
URL = "https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifList.do?pageUnit=60&pageIndex="
policy_id_list = get_ids_with_state(59, URL)
# 상태 메시지
print(f"{len(policy_id_list)}개 크롤링 완료")

1342개 크롤링 완료


In [3]:
policy_id_list

[['R2024011018628', '국토교통부'],
 ['R2024021319564', '국토교통부'],
 ['R2024010218403', '고용노동부'],
 ['R2024030520303', '보건복지부'],
 ['R2024010818565', '금융위원회'],
 ['R2024020619508', '광주'],
 ['R2024010400002', '국토교통부'],
 ['R2024020619531', '국세청'],
 ['R2024010218423', '고용노동부'],
 ['R2024020719543', '고용노동부'],
 ['R2024020619507', '중소벤처기업진흥공단'],
 ['R2024052323266', '보건복지부'],
 ['R2024031820743', '고용노동부'],
 ['R2024011919045', '서울'],
 ['R2024031920824', '서울'],
 ['R2024010818569', '국세청'],
 ['R2024022720164', '국토교통부'],
 ['R2024010918589', '국토교통부'],
 ['R2024011118690', '고용노동부'],
 ['R2024010918590', '국토교통부'],
 ['R2024031320643', '보건복지부'],
 ['R2024032521044', '고용노동부'],
 ['R2024011618762', '한국토지주택공사'],
 ['R2024041621858', '강원'],
 ['R2024011618877', '인천'],
 ['R2024032020863', '서울'],
 ['R2024021419627', '보건복지부'],
 ['R2024021319565', '고용노동부'],
 ['R2024021919780', '부산'],
 ['R2024031520703', '보건복지부'],
 ['R2024030420294', '서울'],
 ['R2024011819025', '서울'],
 ['R2024042222043', '문화체육관광부'],
 ['R2024061223884', '고용노동부'],
 

#### 정책 ID list 중간 저장

- `txt` 형식으로 `policy_id_list`저장


In [1]:
import json
import os
import pickle


def save_pickle(path: str, file_name: str, file: list):
    """
    정책 id list를 받아 pickle 형식의 파일로 저장하는 함수입니다.
    Parameters:
        path (str): 저장할 루트
        file_name (str): 파일명
        file (list[list]): 저장할 정책 id list [id, organ_name]
    Notes:
        경로 폴더를 만들지 않았더라도 os.makedirs를 사용해서 자동으로 생성해줍니다.
        exist_ok를 True로 설정하여 혹 경로 폴더가 있어도 적용됩니다.
        한줄에 한 id, organ_name씩 나오게 했습니다.
    """
    os.makedirs(path, exist_ok=True)
    with open(path + "/" + file_name, "wb") as f:
        pickle.dump(policy_id_list, f)


def load_pickle(path: str):
    """
    pickle 형식의 파일을 불러오는 함수입니다.
    Parameters:
        path (str): 불러올 루트
    Returns:
        data (list): 불러온 데이터
    """
    with open(path, "rb") as f:
        policy_id_list = pickle.load(f)
    return policy_id_list


def save_json(path: str, file_name: str, file: list):
    """
    정책 list를 받아 json 형식의 파일로 저장하는 함수입니다.
    Parameters:
        path (str): 저장할 루트
        file_name (str): 파일명
        file (list[dict]): 저장할 정책 list
    Notes:
        경로 폴더를 만들지 않았더라도 os.makedirs를 사용해서 자동으로 생성해줍니다.
        exist_ok를 True로 설정하여 혹 경로 폴더가 있어도 적용됩니다.
        encoding은 utf-8로 적용했습니다.
        indent를 4를 주어 가독성이 좋게 저장합니다.
        ensure_ascii를 False를 주어 한글을 전부 표현할 수 있게 합니다.
    """
    os.makedirs(path, exist_ok=True)
    with open(path + "/" + file_name, "w", encoding="utf-8") as f:
        json.dump(file, f, indent=4, ensure_ascii=False)


def load_json(path: str):
    """
    json 형식의 파일을 불러오는 함수입니다.
    Parameters:
        path (str): 불러올 루트
    Returns:
        data (list): 불러온 데이터
    """
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

In [4]:
# 저장
save_pickle("data/raw", "policy_id_list.pkl", policy_id_list)

In [6]:
# 불러오기
policy_id_list = load_pickle("data/raw/policy_id_list.pkl")

In [7]:
policy_id_list

[['R2024011018628', '국토교통부'],
 ['R2024021319564', '국토교통부'],
 ['R2024010218403', '고용노동부'],
 ['R2024030520303', '보건복지부'],
 ['R2024010818565', '금융위원회'],
 ['R2024020619508', '광주'],
 ['R2024010400002', '국토교통부'],
 ['R2024020619531', '국세청'],
 ['R2024010218423', '고용노동부'],
 ['R2024020719543', '고용노동부'],
 ['R2024020619507', '중소벤처기업진흥공단'],
 ['R2024052323266', '보건복지부'],
 ['R2024031820743', '고용노동부'],
 ['R2024011919045', '서울'],
 ['R2024031920824', '서울'],
 ['R2024010818569', '국세청'],
 ['R2024022720164', '국토교통부'],
 ['R2024010918589', '국토교통부'],
 ['R2024011118690', '고용노동부'],
 ['R2024010918590', '국토교통부'],
 ['R2024031320643', '보건복지부'],
 ['R2024032521044', '고용노동부'],
 ['R2024011618762', '한국토지주택공사'],
 ['R2024041621858', '강원'],
 ['R2024011618877', '인천'],
 ['R2024032020863', '서울'],
 ['R2024021419627', '보건복지부'],
 ['R2024021319565', '고용노동부'],
 ['R2024021919780', '부산'],
 ['R2024031520703', '보건복지부'],
 ['R2024030420294', '서울'],
 ['R2024011819025', '서울'],
 ['R2024042222043', '문화체육관광부'],
 ['R2024061223884', '고용노동부'],
 

##### 정책 상세 크롤링

- 참고용 index(`list_tit`과 `list_cont`)

| index | name               | index | name                     |
| ----- | ------------------ | ----- | ------------------------ |
| 0     | "정책 번호"        | 13    | "추가 단서 사항"         |
| 1     | "정책 분야"        | 14    | "참여 제한 대상"         |
| 2     | "지원 내용"        | 15    | "신청 절차"              |
| 3     | "사업 운영 기간"   | 16    | "심사 및 발표"           |
| 4     | "사업 신청청 기간" | 17    | "신청 사이트"            |
| 5     | "지원 규모(명)"    | 18    | "제출 서류"              |
| 6     | "비고"             | 19    | "기타 유익 정보"         |
| 7     | "연령"             | 20    | "주관 기관"              |
| 8     | "거주지 및 소득"   | 21    | "운영 기관"              |
| 9     | "학력"             | 22    | "사업관련 참고 사이트 1" |
| 10    | "전공"             | 23    | "사업관련 참고 사이트 2" |
| 11    | "취업 상태"        | 24    | "첨부파일"               |
| 12    | "특화 분야"        |

- `객체.text`로 추출했을 때, 추출되지 않는 요소들이 있어 `객체.contents`로 개별 요소 추출.
  - 요소에 `<br/>`이 있는 경우, 이 현상이 잘 나타남. -> `<br/>`태그 제거
  - `객체.text`로 추출한 경우 과도한 띄어쓰기 및 줄바꿈 발견
  - `객체.contents`로 추출한 경우 과도한 띄어쓰기 및 줄바꿈이 이스케이프 문자로 추출 -> `\n, \t`등 이스케이프 문자 제거


In [8]:
# 불러오기
with open("data/raw/policy_id_list.pkl", "rb") as f:
    policy_id_list = pickle.load(f)

In [9]:
# 문자열 처리 함수 정의(html 태그 제거, 이스케이프 문자 제거, 과도한 띄어쓰기 제거)
import re


def formated(string):
    """
    문자열을 받아서 html 태그와 이스케이프 문자, 과도한 띄어쓰기를 제거하는 함수입니다.
    Parameters:
        string (str): 처리할 문자열
    Returns:
        string (str): 처리된 문자열
    Notes:
        여기서 과도한 띄어쓰기란 두 개 이상의 띄어쓰기를 한 경우를 뜻합니다.
        전체 띄어쓰기를 한 칸으로 맞추려면 시간이 오래 소요되기 때문에 두 칸만 제거합니다.
    """
    tag_format = r"\<.*?\>"
    string = string.replace("\n", "")
    string = string.replace("\t", "")
    string = re.sub(tag_format, "", str(string))
    string = string.replace("  ", "")
    return string

In [10]:
# 상세 내용 크롤링 함수 정의
import requests
from bs4 import BeautifulSoup as bs
import re


def crawling(
    policy_id_list: list, url: str, params: dict, cont_attrs: bool = True | False
):
    """
    정책 id list와 url을 넣으면 dictionary 형태로 크롤링하여 반환하는 함수입니다.
    Parameters:
        policy_id_list (list): 앞서 추출한 정책 id list입니다.
        url (str): 추출 대상의 페이지 링크를 조합할 베이스 링크입니다.
                    url은 반드시 제일 뒤에 정책 id가 들어갈 수 있는 형태로 넣어야 합니다.
                    url=https://www.OOO.OOO/BizID=
        params (dict): 추출 대상의 제목, 항목, 내용을 추출할 html 태그를 dictionary 형식으로 전달합니다.
                    params={"title": [name, attr],
                            "list_tit":[name, attr],
                            "list_cont":[name, attr]}
        cont_attrs (bool): 추출 대상의 항목과 내용이 class나 id 같은 속성 없이 html 태그만으로 추출이 가능하다면 False를
                            속성이 필요하다면 True를 선택합니다.(Default: True)
    Returns:
        total_policy (list[dict]): 추출된 정책을 항목: 내용의 dictionary 형식으로 만들어 전체 정책을 list로 반환합니다.
    """
    total_policy = []
    format = {"br": r"<br/>", "a": r"<a href"}
    for id, organ in policy_id_list:
        policy = {}
        URL = url
        URL += id
        # request로 호출(간혹 https로 호출이 안되는 타 사이트가 있어, 예외가 발생한 경우 http로 url를 변경하여 다시 시도)
        try:
            response = requests.get(URL)
        except:
            URL.replace("https", "http")
            response = requests.get(URL)
        soup = bs(response.text, "html.parser")
        # 정책 이름 추출
        title = soup.find(params["title"][0], params["title"][1]).text
        policy["정책 이름"] = title
        # 항목, 내용 list 추출(같은 index로 연결)
        if cont_attrs:
            policy["기관"] = organ
            subtitle = soup.find("p", "doc_desc").text
            subtitle = subtitle.replace("\r", " ")
            subtitle = subtitle.strip()
            policy["요약"] = subtitle
            list_tit = soup.find_all(
                name=params["list_tit"][0], attrs=params["list_tit"][1]
            )
            list_cont = soup.find_all(
                name=params["list_cont"][0], attrs=params["list_cont"][1]
            )
        else:
            list_tit = soup.find_all(name=params["list_tit"][0])
            list_cont = soup.find_all(name=params["list_cont"][0])
        # 항목 내용 처리
        for i in range(len(list_tit)):
            # list_cont[i].contents = ["\n", "ㅁㅁㅁ", "\n"] 또는 ["\n\t\t\t\tㅁㅁㅁㅁ\n\t\t\t\t", "<br/>", "ㅁㅁㅁ"] 이런 식으로 나옴
            if len(list_cont[i].contents) > 1:
                contents = []
                for j in range(len(list_cont[i].contents)):
                    content = list_cont[i].contents[j]
                    # <br/> 제거
                    if re.match(format["br"], str(content)) != None:
                        content = None
                    # url만 있는 경우 추출
                    elif re.match(format["a"], str(content)) != None:
                        content = content.attrs["href"]
                    # 그 외 공백 제거, '\n', '\t', 제거 안된 html 태그 제거
                    else:
                        content = content.text
                        content = content.strip()
                        content = formated(content)
                    # 처리 작업이 끝난 후 의미있는 요소만 contents(list)에 추가
                    if content not in [None, "\n", "", ","]:
                        # \r이 있을 경우 이를 구분자로 분할한 뒤 삽입
                        if "\r" in content:
                            content = content.split("\r")
                            for con in content:
                                contents.append(con)
                        else:
                            contents.append(content)
                if len(contents) == 1:
                    contents = "".join(contents)
            else:
                contents = list_cont[i].contents
                contents = "".join(contents)
                contents = formated(contents)

            # 동일한 요소가 contents(list)에 들어있을 경우
            if (
                isinstance(contents, list)
                and len(contents) == 2
                and contents[0] == contents[1]
            ):
                contents = set(contents)
                contents = "".join(contents)
                contents = formated(contents)
            # 정책의 항목 이름, 내용 연결
            policy[list_tit[i].text] = contents
        total_policy.append(policy)
    return total_policy

In [11]:
# 온통청년 정책 크롤링
total_policy = []
URL = "https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?bizId="
params = {
    "title": ["h2", "doc_tit01 type2"],
    "list_tit": ["div", "list_tit"],
    "list_cont": ["div", "list_cont"],
}
total_policy = crawling(policy_id_list, URL, params, cont_attrs=True)

In [12]:
total_policy

[{'정책 이름': '청년 주택드림 청약통장',
  '기관': '국토교통부',
  '요약': '청년의 내 집 마련을 위해 장기,저리 대출 지원',
  '정책 번호': 'R2024011018628',
  '정책 분야': '주거분야',
  '지원 내용': ['□ 이자율: 최대 4.5%',
   '□ 소득기준: 연 5,000만원(직전년도 신고소득이 있는 자 기준), 무주택자',
   '□ 납입한도: 월 100만원',
   '□ 납입금액의 40%까지 소득공제',
   '□ 주택청약에 당첨된 경우 청년주택드림 대출 연계',
   'ㅇ 청년주택드림 청약통장 1년 이상 가입',
   'ㅇ 1,000만원 이상 납입',
   'ㅇ 당첨 시 분양가 80%까지 대출',
   'ㅇ 금리 최저 연 2.2%(만기, 소득별 차등) 최장 40년까지 지원(고정금리)',
   'ㅇ 6억 이하, 전용면적 85제곱미터 이하 주택',
   'ㅇ 생애주기별 금리 인하: 결혼 0.1%p 인하, 출산: 0.5%p 인하, 다자녀: 0.2%p 인하'],
  '사업 운영 기간': '-',
  '사업 신청 기간': '2024년 02월21일 ~ 2024년 12월31일',
  '지원 규모(명)': '',
  '비고': '※기존 청년우대형통장 가입자는 출시일에 맞춰서 자동 전환 일반청약 가입자는 요건에만 맞으면 전환 가입 가능',
  '연령': '만 19세 ~ 34세',
  '거주지 및 소득': '□ 청년 주택드림 청약통장 소득 기준: 연 5,000만원 이하',
  '학력': '제한없음',
  '전공': '제한없음',
  '취업 상태': '제한없음',
  '특화 분야': '제한없음',
  '추가 단서 사항': '',
  '참여 제한 대상': '',
  '신청 절차': '',
  '심사 및 발표': '',
  '신청 사이트': '',
  '제출 서류': '',
  '기타 유익 정보': ['□ 업무 취급 은행',
   '○ 우리은행 : 1599-0800',
   '○ KB 국민은행 : 1599-1771',
   '○ 

##### 정책 json 파일로 저장


In [13]:
save_json("data/raw", "policy.json", total_policy)